# Retrieve movie data from the MovieLens project and TMDB

In [1]:
import pandas as pd
import numpy as np
import requests
import os
from IPython.display import display
import urllib.request
import time
import shutil

import random
random.seed(1)

In [ ]:
tmdb_api_key = None
assert tmdb_api_key is not None , 'Error: please set tmdb api key'

## Load MovieLens Data

In [2]:
if not os.path.exists(os.path.join('data/ml-20m')):

    import urllib.request
    urllib.request.urlretrieve('http://files.grouplens.org/datasets/movielens/ml-20m.zip', 'ml-20m.zip')

    import zipfile
    with zipfile.ZipFile('ml-20m.zip',"r") as zip_ref:
        zip_ref.extractall()
        
    os.remove('ml-20m.zip')
    shutil.move("ml-20m", os.path.join('data', 'ml-20m'))

## Load movie posters, year of release and language from TMDB

The MovieLens data doesn't contain the **movie posters, year of release or language**. This means we have to gather this information from other resources. Fortunately the MovieLens data contains the ids in [tmdb](https://www.themoviedb.org/) for each move. Tmdb is a movie database with a plethora of information about each movie and fortunatelly one can access this data through an API. 

### Merge general movie data and tmdb ids

In [3]:
movies = pd.read_csv(os.path.join('data', 'ml-20m', 'movies.csv'), index_col='movieId')
display(movies.head(3))
movies.shape

,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance


(27278, 2)

In [4]:
links = pd.read_csv(os.path.join('data', 'ml-20m', 'links.csv'), index_col='movieId')
display(links.head(3))
links.shape

,imdbId,tmdbId
movieId,,
1,114709,862.0
2,113497,8844.0
3,113228,15602.0


(27278, 2)

In [5]:
tmdb_links = links.tmdbId
tmdb_links.dropna(inplace=True)
tmdb_links = tmdb_links.astype('int32')
display(tmdb_links.head(3))
tmdb_links.shape

movieId
1      862
2     8844
3    15602
Name: tmdbId, dtype: int32

(27026,)

In [6]:
movies = movies.join(tmdb_links, how='inner')
display(movies.head(3))
movies.shape

,title,genres,tmdbId
movieId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862
2,Jumanji (1995),Adventure|Children|Fantasy,8844
3,Grumpier Old Men (1995),Comedy|Romance,15602


(27026, 3)

### Retrieve language, year of release and a link to a movie poster

The tmdb API allows only a 40 calls every 10s. For a dataset of around 27000 movies the retrieval is going to take a couple hours. Therefore we have to cache the intermediate results every now and then.

In [7]:

extended_movie_data_path = os.path.join('data', 'extended_movie_data.csv')
#Extend the movies dataframe by columns for language, year of release, and link to the movie poster and cache the result
if not os.path.exists(extended_movie_data_path):
    movies['release_year'] = ''
    movies['poster_url'] = ''
    movies['language'] = ''
    movies.to_csv(extended_movie_data_path, sep=';')
    
movies = pd.read_csv(extended_movie_data_path, sep=';', index_col='movieId', keep_default_na=False)
display(movies.head(3))
movies.shape

,title,genres,tmdbId,release_year,poster_url,language
movieId,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862,1995,https://image.tmdb.org/t/p/w300/rhIRbceoE9lR4v...,en
2,Jumanji (1995),Adventure|Children|Fantasy,8844,1995,https://image.tmdb.org/t/p/w300/vgpXmVaVyUL7GG...,en
3,Grumpier Old Men (1995),Comedy|Romance,15602,1995,https://image.tmdb.org/t/p/w300/6ksm1sjKMFLbO7...,en


(27026, 6)

In [10]:
from dateutil.parser import parse

tmdb_movie_url = 'https://api.themoviedb.org/3/movie/{}?api_key=' + tmdb_api_key
poster_base_url = 'https://image.tmdb.org/t/p/w300{}'

# Retrieves language, year of release and a link to a movie poster for a given tmdb id throuth the tmdb API. Returns 'not found' \
# if data couldn't be retrieved
def get_more_movie_data(tmdbId):
    
    year = 'not found'
    language = 'not found'
    poster_url = 'not found'
    
    try:
        response = requests.get(tmdb_movie_url.format(tmdbId))
    except:
        print('Data {} couldn\'t be retrieved'.format(tmdbId))
        return year, poster_url, language
    
    tmdb_movie_json = response.json()
    
    try:
        date_string = tmdb_movie_json['release_date']
    
        try:
            dt = parse(date_string)
            year = dt.year
        except:
            print(tmdb_movie_json)
            print('Date {} can\'t be parsed'.format(date_string))
    except:
        print('release_data was not found in response \n {}'.format(tmdb_movie_json))
    
    try:
        language = tmdb_movie_json['original_language']
    except:
        print('original_language was not found in response \n {}'.format(tmdb_movie_json))
    
    try:
        if poster_path in tmdb_movie_json and tmdb_movie_json['poster_path'] is not None:
            poster_url = poster_base_url.format(tmdb_movie_json['poster_path'])
        else:
            print('Poster_path {} can\'t be parsed'.format(poster_path))       
    except:
        print('poster_path was not found in response \n {}'.format(tmdb_movie_json))
    
    return year, poster_url, language
    

In [11]:
count = 0

for index, row in movies.iterrows():
    count += 1
    if row['release_year'] == '':
        time.sleep(0.5)
        release_year, poster_url, language = get_more_movie_data(row.tmdbId)
        
        movies.at[index, 'release_year'] = release_year
        movies.at[index, 'poster_url'] = poster_url
        movies.at[index, 'language'] = language

        if count % 20 == 0:
            print('{} movies extended'.format(count))
            movies.to_csv(extended_movie_data_path, sep=';')
    else:
        if count % 20 == 0:
            print('{} movies extended'.format(count))
 

movies.to_csv(extended_movie_data_path, sep=';')

20 movies extended
40 movies extended
60 movies extended
80 movies extended
100 movies extended
120 movies extended
140 movies extended
160 movies extended
180 movies extended
200 movies extended
220 movies extended
240 movies extended
260 movies extended
280 movies extended
300 movies extended
320 movies extended
340 movies extended
360 movies extended
380 movies extended
400 movies extended
420 movies extended
440 movies extended
460 movies extended
480 movies extended
500 movies extended
520 movies extended
540 movies extended
560 movies extended
580 movies extended
600 movies extended
620 movies extended
640 movies extended
660 movies extended
680 movies extended
700 movies extended
720 movies extended
740 movies extended
760 movies extended
780 movies extended
800 movies extended
820 movies extended
840 movies extended
860 movies extended
880 movies extended
900 movies extended
920 movies extended
940 movies extended
960 movies extended
980 movies extended
1000 movies extended
102

8500 movies extended
8520 movies extended
8540 movies extended
8560 movies extended
8580 movies extended
8600 movies extended
8620 movies extended
8640 movies extended
8660 movies extended
8680 movies extended
8700 movies extended
8720 movies extended
8740 movies extended
8760 movies extended
8780 movies extended
8800 movies extended
8820 movies extended
8840 movies extended
8860 movies extended
8880 movies extended
8900 movies extended
8920 movies extended
8940 movies extended
8960 movies extended
8980 movies extended
9000 movies extended
9020 movies extended
9040 movies extended
9060 movies extended
9080 movies extended
9100 movies extended
9120 movies extended
9140 movies extended
9160 movies extended
9180 movies extended
9200 movies extended
9220 movies extended
9240 movies extended
9260 movies extended
9280 movies extended
9300 movies extended
9320 movies extended
9340 movies extended
9360 movies extended
9380 movies extended
9400 movies extended
9420 movies extended
9440 movies e

18120 movies extended
18140 movies extended
18160 movies extended
18180 movies extended
18200 movies extended
18220 movies extended
18240 movies extended
18260 movies extended
18280 movies extended
18300 movies extended
18320 movies extended
18340 movies extended
18360 movies extended
18380 movies extended
18400 movies extended
18420 movies extended
18440 movies extended
18460 movies extended
18480 movies extended
18500 movies extended
18520 movies extended
18540 movies extended
18560 movies extended
18580 movies extended
18600 movies extended
18620 movies extended
18640 movies extended
18660 movies extended
18680 movies extended
18700 movies extended
18720 movies extended
18740 movies extended
18760 movies extended
18780 movies extended
18800 movies extended
18820 movies extended
18840 movies extended
18860 movies extended
18880 movies extended
18900 movies extended
18920 movies extended
18940 movies extended
18960 movies extended
18980 movies extended
19000 movies extended
19020 movi

25680 movies extended
25700 movies extended
25720 movies extended
25740 movies extended
25760 movies extended
25780 movies extended
25800 movies extended
25820 movies extended
25840 movies extended
25860 movies extended
25880 movies extended
25900 movies extended
25920 movies extended
25940 movies extended
25960 movies extended
25980 movies extended
26000 movies extended
26020 movies extended
26040 movies extended
26060 movies extended
26080 movies extended
26100 movies extended
26120 movies extended
26140 movies extended
26160 movies extended
26180 movies extended
26200 movies extended
26220 movies extended
26240 movies extended
26260 movies extended
26280 movies extended
26300 movies extended
26320 movies extended
26340 movies extended
26360 movies extended
26380 movies extended
26400 movies extended
26420 movies extended
26440 movies extended
26460 movies extended
26480 movies extended
26500 movies extended
26520 movies extended
26540 movies extended
26560 movies extended
26580 movi

### Retrieve images of movie posters

In [12]:
extended_movie_data_with_local_files_path = os.path.join('extended_movie_data_with_local_files.csv')
# Extend the movie dataframe by a new column with the file name of the respective movie poster.
if not os.path.exists(extended_movie_data_with_local_files_path):
    movies['local_poster_file'] = ''
    movies.to_csv(extended_movie_data_with_local_files_path, sep=';')

movies = pd.read_csv(extended_movie_data_with_local_files_path, sep=';', index_col='movieId', keep_default_na=False)
display(movies.head(3))
movies.shape

,title,genres,tmdbId,release_year,poster_url,language,local_poster_file
movieId,,,,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,862,1995,https://image.tmdb.org/t/p/w300/rhIRbceoE9lR4v...,en,
2,Jumanji (1995),Adventure|Children|Fantasy,8844,1995,https://image.tmdb.org/t/p/w300/vgpXmVaVyUL7GG...,en,
3,Grumpier Old Men (1995),Comedy|Romance,15602,1995,https://image.tmdb.org/t/p/w300/6ksm1sjKMFLbO7...,en,


(27026, 7)

In [13]:
# Downloads and saves a movie poster in the 'posters' directory for a given link to the movie poster
def save_poster(movieId, poster_url):
    poster_response = None
    retries = 0
    while poster_response is None and retries < 5:
        retries += 1
        try:
            poster_response = requests.get(poster_url, stream=True)
        except:
            print('Poster {} couldn\'t be retrieved. Retry {}'.format(poster_url, retries))
        
        if poster_response is not None:
            local_poster_file = str(movieId) + '.' + poster_url.split('.')[-1]
            poster_save_path = os.path.join('posters', local_poster_file)
            with open(poster_save_path, 'wb') as out_file:
                shutil.copyfileobj(poster_response.raw, out_file)
            return local_poster_file
    return 'not found' 
       
count = 0
for index, row in movies.iterrows():
    count += 1
    poster_url = row.poster_url
    if row.local_poster_file == '':
        local_poster_file = save_poster(index, poster_url)
        movies.at[index, 'local_poster_file'] = local_poster_file 

        if count % 10 == 0:
            print('{} posters saved'.format(count))
            movies.to_csv(extended_movie_data_with_local_files_path, sep=';')
    
    else:
        if count % 100 == 0:
            print('{} posters saved'.format(count))

movies.to_csv(extended_movie_data_with_local_files_path, sep=';')                 

10 posters saved
20 posters saved
30 posters saved
40 posters saved
50 posters saved
60 posters saved
70 posters saved
80 posters saved
90 posters saved
100 posters saved
110 posters saved
120 posters saved
130 posters saved
140 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
150 posters saved
160 posters saved
170 posters saved
180 posters saved
190 posters saved
200 posters saved
210 posters saved
220 posters saved
230 posters saved
240 posters saved
250 posters saved
260 posters saved
270 posters saved
280 posters saved
290 posters saved
300 posters saved
310 posters saved
320 posters saved
330 posters saved
340 posters saved
350 posters saved
360 posters saved
370 posters saved
380 posters saved
390 posters saved
400 posters saved
410 posters saved
420 posters saved
430 posters s

3110 posters saved
3120 posters saved
3130 posters saved
3140 posters saved
3150 posters saved
3160 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
3170 posters saved
3180 posters saved
3190 posters saved
3200 posters saved
3210 posters saved
3220 posters saved
3230 posters saved
3240 posters saved
3250 posters saved
3260 posters saved
3270 posters saved
3280 posters saved
3290 posters saved
3300 posters saved
3310 posters saved
3320 posters saved
3330 posters saved
3340 posters saved
3350 posters saved
3360 posters saved
3370 posters saved
3380 posters saved
3390 posters saved
3400 posters saved
3410 posters saved
3420 posters saved
3430 posters saved
3440 posters saved
3450 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2

6040 posters saved
6050 posters saved
6060 posters saved
6070 posters saved
6080 posters saved
6090 posters saved
6100 posters saved
6110 posters saved
6120 posters saved
6130 posters saved
6140 posters saved
6150 posters saved
6160 posters saved
6170 posters saved
6180 posters saved
6190 posters saved
6200 posters saved
6210 posters saved
6220 posters saved
6230 posters saved
6240 posters saved
6250 posters saved
6260 posters saved
6270 posters saved
6280 posters saved
6290 posters saved
6300 posters saved
6310 posters saved
6320 posters saved
6330 posters saved
6340 posters saved
6350 posters saved
6360 posters saved
6370 posters saved
6380 posters saved
6390 posters saved
6400 posters saved
6410 posters saved
6420 posters saved
6430 posters saved
6440 posters saved
6450 posters saved
6460 posters saved
6470 posters saved
6480 posters saved
6490 posters saved
6500 posters saved
6510 posters saved
6520 posters saved
6530 posters saved
6540 posters saved
6550 posters saved
6560 posters

8930 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
8940 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
8950 posters saved
8960 posters saved
8970 posters saved
8980 posters saved
8990 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
9000 posters saved
9010 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not fo

Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
10430 posters saved
10440 posters saved
10450 posters saved
10460 posters saved
10470 posters saved
10480 posters saved
10490 posters saved
10500 posters saved
10510 posters saved
10520 posters saved
10530 posters saved
10540 posters saved
10550 posters saved
10560 posters saved
10570 posters saved
10580 posters saved
10590 posters saved
10600 posters saved
10610 posters saved
10620 posters saved
10630 posters saved
10640 posters saved
10650 posters saved
10660 posters saved
10670 posters saved
10680 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5


...

21380 posters saved
21390 posters saved
21400 posters saved
21410 posters saved
21420 posters saved
21430 posters saved
21440 posters saved
21450 posters saved
21460 posters saved
21470 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
21480 posters saved
21490 posters saved
21500 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
21510 posters saved
21520 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be

22580 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
22590 posters saved
22600 posters saved
22610 posters saved
22620 posters saved
22630 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
22640 posters saved
22650 posters saved
22660 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
22670 posters saved
22680 posters saved
22690 posters saved
22700 posters saved
22710 posters saved


24140 posters saved
24150 posters saved
24160 posters saved
24170 posters saved
24180 posters saved
24190 posters saved
24200 posters saved
24210 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
24220 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
24230 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be ret

25160 posters saved
25170 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
25180 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
25190 posters saved
25200 posters saved
25210 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
25220 posters saved
25230 posters saved
25240 posters saved
25250 posters saved
25260 posters saved
25270 posters saved
25280 posters saved
Poster not found cou

25970 posters saved
25980 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
25990 posters saved
26000 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
26010 posters saved
26020 posters saved
26030 posters saved
26040 posters saved
26050 posters saved
26060 posters saved
26070 posters saved
26080 posters saved
Poster not found couldn't be retrieved. Retry 1
Poster not found couldn't be retrieved. Retry 2
Poster not found couldn't be retrieved. Retry 3
Poster not found couldn't be retrieved. Retry 4
Poster not found couldn't be retrieved. Retry 5
26090 posters saved
26100 posters saved
